# Prepare mean teacher dataset

* done: exclude images with masks that have no roi

remove dots for replace:
E:\Christina.\
C:\snec_data.\

In [1]:
import os
import pandas as pd
from PIL import Image
import numpy as np

run_ichallenge_amd  = True
run_ichallenge_glaucoma = False

run_plex = False
run_cirrus = False
run_testset = True



In [9]:
def plex_generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=None, mask_crop=False, drop=False):
    
    # have to call for each dataset that contains images + masks
    # finding image (.image_postfix) and mask (.mask_postfix) pairs for same base filename
    # this works if your_img_dir/A0001.jpg and your_mask_dir/A0001.bmp
    # optic nerve head
    # converts all the masks - 0 background, 255 foreground if new_mask_path is not None
    
    mode = []
    
    import glob
    # need one file for each concept 
    image_path = glob.glob(images_path) # list(set(os.listdir(images_path)) - {'desktop.ini'})
    
    
    image_id = [os.path.basename(x).replace(image_postfix, "") for x in image_path]
    
    
    
    print(image_id[0:5])
    
    image_path = [os.path.join(images_path, x) for x in image_path]
    a = {"image_path" : image_path, "id" : image_id}
    a = pd.DataFrame(data={"img_path" : image_path}, index=image_id)
    
    has_roi = []
    
    if masks_path is not None:
        mask_path  = glob.glob(masks_path)
        mask_id = [os.path.basename(x).replace(mask_postfix, "").replace("mask_", "").replace("DiskMask", "") for x in mask_path]
        mask_path = [os.path.join(masks_path, x) for x in mask_path]
                
        # b = {"msk_path" : msk_path, "id" : mask_id, "has_roi" : has_roi}
        b = pd.DataFrame(data={"msk_path" : mask_path}, index=mask_id)
        
        
        print(a.head())
        print(b.head())
        
        csv_data = pd.concat([a, b], axis=1, join="outer")
        
        csv_data["has_mask"] = [True] * len(a.index) # this has a bug
        csv_data["has_roi"] = [False] * len(a.index) # this too
        
        
    else: # if no mask available
        csv_data = a
        csv_data["msk_path"] = [None] * len(a.index)
        csv_data["has_mask"] = [False] * len(a.index)
        csv_data["has_roi"] = [None] * len(a.index)
        
    
    csv_data = csv_data.sample(frac=1, random_state=19)

    percent_train = int(len(csv_data)/100*split_percentage)
    percent_val = len(csv_data)-percent_train
    
    mode.extend(["train" for i in range(percent_train)])
    mode.extend(["val" for i in range(percent_val)])
    
    
    csv_data["mode"] = mode
    csv_data["weight"] = [weight]*len(mode)
    csv_data["mask_crop"] = [mask_crop]*len(mode)

    print(csv_data)
    
    if drop:
        csv_data = csv_data.dropna(subset=['msk_path'])
    
    csv_data.to_csv(csv_path, sep=';')
            

In [10]:
if run_testset:
    images_path = r"E:\Christina\Result_Data\CIRRUS_Glaucoma\enface\*_cube_z.tif"
    masks_path  = r"E:\Christina\Result_Data\CIRRUS_Glaucoma\enface_mask_manual\*.png" # already ROI in white
    csv_path    = r"test_data_cirrus.csv"
    weight = 1
    plex_generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".tif", mask_postfix=".png", split_percentage=80, new_mask_path=None, drop=True)

['enface_PSNASERI0001_Angiography 6x6 mm_6-29-2018_11-55-24_OD_sn0010_cube_z', 'enface_PSNASERI0003_Angiography 6x6 mm_7-3-2018_10-20-22_OD_sn0025_cube_z', 'enface_PSNASERI0003_Angiography 6x6 mm_7-3-2018_10-24-16_OS_sn0029_cube_z', 'enface_PSNASERI0004_Angiography 6x6 mm_7-4-2018_10-31-7_OD_sn0032_cube_z', 'enface_PSNASERI0004_Angiography 6x6 mm_7-4-2018_10-32-5_OD_sn0033_cube_z']
                                                                                             img_path
enface_PSNASERI0001_Angiography 6x6 mm_6-29-201...  E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
enface_PSNASERI0003_Angiography 6x6 mm_7-3-2018...  E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
enface_PSNASERI0003_Angiography 6x6 mm_7-3-2018...  E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
enface_PSNASERI0004_Angiography 6x6 mm_7-4-2018...  E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
enface_PSNASERI0004_Angiography 6x6 mm_7-4-2018...  E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...
   

In [3]:
if run_plex:
    images_path = r"E:\Christina\Result_Data\PLEX_Healthy\enface\*\*_cube_z.tif"
    masks_path  = r"E:\Christina\Result_Data\PLEX_Healthy\enface_mask\*.png" # already ROI in white
    csv_path    = r"mt_data_plex.csv"
    weight = 1
    plex_generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".tif", mask_postfix=".png", split_percentage=80, new_mask_path=None)

['PE_GLAU0199_Angio (6mmx6mm)_6-10-2021_12-51-6_OD_sn1373_cube_z', 'PE_GLAU0199_Angio (6mmx6mm)_6-10-2021_12-59-34_OS_sn1379_cube_z', 'PR1406_PS0015_Angio (6mmx6mm)_6-17-2021_15-53-38_OD_sn3401_cube_z', 'PR1406_PS0032-C_Angio (6mmx6mm)_7-28-2021_15-21-26_OD_sn3508_cube_z', 'PR1406_PS0048-S_Angio (6mmx6mm)_8-19-2021_11-27-54_OS_sn3610_cube_z']
                                                                                             img_path
PE_GLAU0199_Angio (6mmx6mm)_6-10-2021_12-51-6_O...  E:\Christina\Result_Data\PLEX_Healthy\enface\P...
PE_GLAU0199_Angio (6mmx6mm)_6-10-2021_12-59-34_...  E:\Christina\Result_Data\PLEX_Healthy\enface\P...
PR1406_PS0015_Angio (6mmx6mm)_6-17-2021_15-53-3...  E:\Christina\Result_Data\PLEX_Healthy\enface\P...
PR1406_PS0032-C_Angio (6mmx6mm)_7-28-2021_15-21...  E:\Christina\Result_Data\PLEX_Healthy\enface\P...
PR1406_PS0048-S_Angio (6mmx6mm)_8-19-2021_11-27...  E:\Christina\Result_Data\PLEX_Healthy\enface\P...
                                           

In [4]:
if run_cirrus:
    images_path = r"E:\Christina\Result_Data\CIRRUS_Glaucoma\enface\*_cube_z.tif" # C:\snec_data\Result_Data
    csv_path    = r"mt_data_cirrus.csv"
    weight = 1
    plex_generate_semisupseg_csv_file(images_path, None, csv_path, weight, image_postfix=".tif", mask_postfix=".png", split_percentage=100, new_mask_path=None)

['enface_PSNASERI0001_Angiography 6x6 mm_6-29-2018_11-55-24_OD_sn0010_cube_z', 'enface_PSNASERI0003_Angiography 6x6 mm_7-3-2018_10-20-22_OD_sn0025_cube_z', 'enface_PSNASERI0003_Angiography 6x6 mm_7-3-2018_10-24-16_OS_sn0029_cube_z', 'enface_PSNASERI0004_Angiography 6x6 mm_7-4-2018_10-31-7_OD_sn0032_cube_z', 'enface_PSNASERI0004_Angiography 6x6 mm_7-4-2018_10-32-5_OD_sn0033_cube_z']
                                                                                             img_path  \
enface_PSNASERI0067_Angiography 6x6 mm_8-29-201...  E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...   
enface_PSNASERI0181_Angiography 6x6 mm_12-20-20...  E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...   
enface_PSNASERI0247_Angiography 6x6 mm_3-12-201...  E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...   
enface_PSNASERI0235_Angiography 6x6 mm_3-1-2019...  E:\Christina\Result_Data\CIRRUS_Glaucoma\enfac...   
enface_PSNASERI0032_Angiography 6x6 mm_7-27-201...  E:\Christina\Result_Data\CIRRUS_Glauco

In [ ]:
if run_cirrus:
    images_path = r"E:\Christina\Result_Data\CIRRUS_Normal\enface\*_cube_z.tif" # C:\snec_data\Result_Data
    csv_path    = r"mt_data_cirrus_normal.csv"
    weight = 1
    plex_generate_semisupseg_csv_file(images_path, None, csv_path, weight, image_postfix=".tif", mask_postfix=".png", split_percentage=100, new_mask_path=None)

In [5]:
def iChallenge_generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=None, mask_crop=False):
    
    # this code only works, if ROI is black and background is white
    
    # have to call for each dataset that contains images + masks
    # finding image (.image_postfix) and mask (.mask_postfix) pairs for same base filename
    # this works if your_img_dir/A0001.jpg and your_mask_dir/A0001.bmp
    # optic nerve head
    # converts all the masks - 0 background, 255 foreground if new_mask_path is not None
    
    mode = []
    
    # need one file for each concept 
    image_path = list(set(os.listdir(images_path)) - {'desktop.ini'})
    image_id = [os.path.basename(x).replace(image_postfix, "") for x in image_path]
    image_path = [os.path.join(images_path, x) for x in image_path]
    a = {"image_path" : image_path, "id" : image_id}
    a = pd.DataFrame(data={"img_path" : image_path}, index=image_id)
    
    has_roi = []
    
    if masks_path is not None:
        mask_path  = list(set(os.listdir(masks_path)) - {'desktop.ini'})
        mask_id = [os.path.basename(x).replace(mask_postfix, "") for x in mask_path]
        mask_path = [os.path.join(masks_path, x) for x in mask_path]
        
        new_mask_list = []
            
        for mp in mask_path:

            # all classes = 255 (will become 1), background = 0
            mask = Image.open(mp).convert('L')
            mask = mask.point( lambda p: 0 if p == 255 else 255 )
            mask = mask.convert('1')

            if np.asarray(mask, dtype="int32").sum() == 0:
                has_roi.append(False)
            else:
                has_roi.append(True)
            
            if new_mask_path is not None:
                c = os.path.join(new_mask_path, os.path.basename(mp))
                new_mask_list.append(c)
                mask.save(c)
                #mask.show()
                #break
                #print("unique mask", np.unique(mask))
                #print(self.csv_data.iloc[index]['msk_path'])
                #print("unique mask", np.unique(np.asarray(mask)))


                mask_path = new_mask_list
                
        # b = {"msk_path" : mask_path, "id" : mask_id, "has_roi" : has_roi}
        b = pd.DataFrame(data={"msk_path" : mask_path, "has_roi" : has_roi}, index=mask_id)
        csv_data = pd.concat([a, b], axis=1, join="inner")
        
        csv_data["has_mask"] = [True] * len(a.index)
        
    else: # if no mask available
        csv_data = a
        csv_data["msk_path"] = [None] * len(a.index)
        csv_data["has_mask"] = [False] * len(a.index)
        csv_data["has_roi"] = [None] * len(a.index)
        
    
    csv_data = csv_data.sample(frac=1, random_state=19)

    percent_train = int(len(csv_data)/100*split_percentage)
    percent_val = len(csv_data)-percent_train
    
    mode.extend(["train" for i in range(percent_train)])
    mode.extend(["val" for i in range(percent_val)])
    
    
    csv_data["mode"] = mode
    csv_data["weight"] = [weight]*len(mode)
    csv_data["mask_crop"] = [mask_crop]*len(mode)

    print(csv_data)
    
    # remove images with no roi - problematic in iChallenge, cause some images actually have a partially visible optic disc but it is not annotated
    csv_data = csv_data.drop(csv_data[csv_data.has_roi == False].index)
    
    csv_data.to_csv(csv_path, sep=';')
            

In [6]:
if run_ichallenge_amd:
    images_path =    r"E:\Christina\Result_Data\ADAM_AMD\fundus"
    masks_path  =    r"E:\Christina\Result_Data\ADAM_AMD\fundus_mask"
    new_masks_path = r"E:\Christina\Result_Data\ADAM_AMD\fundus_mask_bin" # where we will save the converted masks
    if not os.path.exists(new_masks_path):
        os.makedirs(new_masks_path)
    csv_path    = r"mt_data_ichallenge_amd.csv"
    weight = 1
    iChallenge_generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=100, new_mask_path=new_masks_path, mask_crop=True)

                                                img_path  \
N0248  E:\Christina\Result_Data\ADAM_AMD\fundus\N0248...   
N0219  E:\Christina\Result_Data\ADAM_AMD\fundus\N0219...   
A0032  E:\Christina\Result_Data\ADAM_AMD\fundus\A0032...   
N0084  E:\Christina\Result_Data\ADAM_AMD\fundus\N0084...   
N0174  E:\Christina\Result_Data\ADAM_AMD\fundus\N0174...   
...                                                  ...   
N0272  E:\Christina\Result_Data\ADAM_AMD\fundus\N0272...   
A0011  E:\Christina\Result_Data\ADAM_AMD\fundus\A0011...   
A0004  E:\Christina\Result_Data\ADAM_AMD\fundus\A0004...   
N0101  E:\Christina\Result_Data\ADAM_AMD\fundus\N0101...   
A0056  E:\Christina\Result_Data\ADAM_AMD\fundus\A0056...   

                                                msk_path  has_roi  has_mask  \
N0248  E:\Christina\Result_Data\ADAM_AMD\fundus_mask_...    False      True   
N0219  E:\Christina\Result_Data\ADAM_AMD\fundus_mask_...    False      True   
A0032  E:\Christina\Result_Data\ADAM_AMD\f

In [7]:
if run_ichallenge_glaucoma:
    images_path =    r"C:\snec_data\Result_Data\REFUGE_Glaucoma\fundus"
    masks_path  =    r"C:\snec_data\Result_Data\REFUGE_Glaucoma\fundus_mask"
    new_masks_path = r"C:\snec_data\Result_Data\REFUGE_Glaucoma\fundus_mask_bin" # where we will save the converted masks
    if not os.path.exists(new_masks_path):
        os.makedirs(new_masks_path)
    csv_path    = r"mt_data_ichallenge_glaucoma.csv"
    weight = 1
    iChallenge_generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=100, new_mask_path=new_masks_path)